## Attendance System using Haar Cascade Classifier

### Sprint 1: Data Collection

In [1]:
import cv2 as cv
import numpy as np

In [2]:
faces = cv.CascadeClassifier(r'C:\Users\LENOVO\Downloads\haarcascade_frontalface_alt.xml')

In [4]:
vid = cv.VideoCapture(0)
cropped = []
count = 0
while True:
    cropped_img=None
    ret, frame = vid.read()
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    face=faces.detectMultiScale(gray_frame, 1.2, 5)
    for (x,y,w,h) in face:
        cropped_img = gray_frame[y-15:y+h+15,x-15:x+h+15]
        if count%20==0:
            cropped.append(cv.resize(cropped_img, (100,100)))
        cv.imshow("Actual", frame)
        cv.imshow("Cropped", cropped_img)
    count+=1
    if cv.waitKey(1) & 0xff == ord('q'):
        break
        
vid.release()
cv.destroyAllWindows()

In [5]:
cropped, len(cropped)

([array([[138, 138, 140, ..., 131, 131, 131],
         [137, 138, 138, ..., 132, 132, 134],
         [137, 138, 140, ..., 135, 133, 134],
         ...,
         [  3,   3,   3, ...,   2,   2,   2],
         [  2,   3,   2, ...,   2,   2,   3],
         [  2,   2,   2, ...,   2,   3,   3]], dtype=uint8),
  array([[163, 164, 164, ..., 157, 158, 158],
         [166, 164, 164, ..., 155, 157, 157],
         [163, 164, 165, ..., 155, 157, 157],
         ...,
         [  9,   7,  16, ...,   2,   2,   2],
         [  7,   8,  14, ...,   2,   2,   2],
         [  3,   5,  13, ...,   2,   2,   2]], dtype=uint8),
  array([[164, 162, 164, ..., 154, 159, 154],
         [161, 164, 166, ..., 156, 157, 156],
         [163, 164, 165, ..., 156, 155, 157],
         ...,
         [ 13,  14,  12, ...,   2,   2,   1],
         [ 13,  12,  13, ...,   2,   2,   2],
         [ 10,   6,   6, ...,   2,   2,   2]], dtype=uint8),
  array([[164, 162, 161, ..., 158, 155, 153],
         [163, 163, 163, ..., 156, 155,

In [6]:
data = np.asarray(cropped)

In [7]:
data.reshape(data.shape[0],-1)

array([[138, 138, 140, ...,   2,   3,   3],
       [163, 164, 164, ...,   2,   2,   2],
       [164, 162, 164, ...,   2,   2,   2],
       ...,
       [161, 165, 164, ...,   2,   0,   5],
       [162, 165, 163, ...,   2,   2,   2],
       [162, 162, 163, ...,   2,   2,   3]], dtype=uint8)

In [7]:
#np.save("Vinathi.npy", data.reshape(data.shape[0],-1))

In [7]:
np.save("Jyothi.npy", data.reshape(data.shape[0],-1))

In [12]:
#np.save("Madhuri.npy", data.reshape(data.shape[0],-1))

In [7]:
#np.save("Jasmine.npy", data.reshape(data.shape[0],-1))

In [8]:
#np.save("Prasanna.npy", data.reshape(data.shape[0],-1))

In [9]:
import numpy as np
import os

# Path to the directory containing the .npy files
path = r'C:\Users\LENOVO\Deep Learnig\OpenCV\OpenCV Project'

final_data = []
count = 0
diction = {}

# Loop through each file in the specified directory
for file in os.listdir(path):
    if file.endswith(".npy"):
        # Load the numpy data (assuming each file contains face data)
        data = np.load(os.path.join(path, file))

        # Create a label array with the same number of rows as the data, and assign the current count as label
        labels = np.full((data.shape[0], 1), count)

        # Concatenate the data with the corresponding labels
        labeled_data = np.concatenate([data, labels], axis=1)

        # Append the labeled data to the final data list
        final_data.append(labeled_data)

        # Optionally, store the label-to-name mapping in a dictionary
        diction[count] = file.replace('.npy', '')  # Removing the extension to use the name as the label

        # Increment the label counter for the next file
        count += 1

# Convert the final_data list into a numpy array
final_data = np.vstack(final_data)

# Output the dictionary for reference
print("Label to Name Mapping: ", diction)

# Save the final concatenated data
np.save('final_data.npy', final_data)


Label to Name Mapping:  {0: 'Jasmine', 1: 'Jyothi', 2: 'Madhuri', 3: 'Prasanna', 4: 'Vinathi'}


### Sprint 2: Building ANN Model to identify the person


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the concatenated data
data = np.load('final_data.npy')

# Separate features and labels
X = data[:, :-1]  # All columns except the last one (features)
y = data[:, -1]   # Last column (labels)

# Normalize the features
X = X / 255.0

# One-hot encode the labels for multi-class classification
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
y_train.shape

(109, 5)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build the ANN model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


C:\Users\LENOVO\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 272ms/step - accuracy: 0.2319 - loss: 1.9479 - val_accuracy: 0.4643 - val_loss: 1.3952
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5844 - loss: 1.1421 - val_accuracy: 0.8214 - val_loss: 0.5635
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9384 - loss: 0.5068 - val_accuracy: 0.8571 - val_loss: 0.3006
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8773 - loss: 0.2895 - val_accuracy: 0.8214 - val_loss: 0.2786
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9249 - loss: 0.1872 - val_accuracy: 1.0000 - val_loss: 0.1189
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 0.0658 - val_accuracy: 1.0000 - val_loss: 0.0748
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 0.0520 - val_accuracy: 1.0000 - val_loss: 0.0627
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.0306 - val_accuracy: 1.0000 - val_loss: 0.0543

In [13]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.0151
Test accuracy: 1.0


In [14]:
# Save the model
model.save('face_recognition_model.h5')


In [3]:
import cv2 as cv
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('face_recognition_model.h5')

# Load Haar Cascade for face detection
face_cascade = cv.CascadeClassifier(r'C:\Users\LENOVO\Downloads\haarcascade_frontalface_alt.xml')

# Create a label-to-name dictionary (adjust as needed)
diction = {0: 'Jasmine', 1: 'Jyothi', 2: 'Madhuri', 3: 'Prasanna', 4: 'Vinathi'}  # Update with your actual label mapping

# Start the webcam for real-time recognition
vid = cv.VideoCapture(0)

while True:
    ret, frame = vid.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract and preprocess face region
        face = gray[y:y+h, x:x+w]
        face_resized = cv.resize(face, (100, 100))
        face_flatten = face_resized.flatten().reshape(1, -1) / 255.0

        # Predict label
        prediction = model.predict(face_flatten)
        label = np.argmax(prediction)

        # Get the name from the dictionary
        name = diction.get(label, "Unknown")

        # Display name on the frame
        cv.putText(frame, name, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Show the frame
    cv.imshow("Real-Time Face Recognition", frame)

    # Exit loop on 'q' key press
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
vid.release()
cv.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━